In [ ]:
import load_data
import feature_attender
from sklearn import linear_model
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth =True

set_session(tf.Session(config=config)) 


'''select  grp(BD orHC)
feature_types is the input features you want
B_1 is Dass-21 
B_2 is Altman
C is daily valence
E is daily 7 emotions
'''
dir_grp = 'HC'
feature_types = ['B_1','B_2','C','D','E']
# get attenders what u  want
attenders = feature_attender.get_wanna_attender(dir_grp,feature_types)

print(dir_grp,'attenders:',len(attenders))
# get input feature and scale (target_scale = 'ymrs' or 'hamd')
# each scale label is 4 dimension (3 factor of YMRS and 1 total score of YMRS)
x, label = load_data.loading_data(dir_grp,attenders,feature_types,target_scale='hamd')
print('res', x.shape, label.shape)

In [ ]:
import keras
from keras.models import Model
from sklearn import linear_model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.linear_model import LassoCV
from keras.utils import plot_model

extract factors

In [ ]:
def neg_zero( L):
    for i in range( len(L)):
        if L[i] < 0:
            L[i] = 0
    return L

In [ ]:
def get_alpha(x_,h_):
    global alpha_
    global check
    x_train = np.nan_to_num( x_)
    y_train = h_
    lassoCV = LassoCV(cv=10)
    lassoCV.fit( x_train, y_train)
    alpha_ = lassoCV.alpha_
    return alpha_

def Lasso_Regression_alpha(X_train,target):
    zero_flag = True
    alpha=get_alpha(X_train,target)
    model = linear_model.Lasso(alpha=alpha)
    model.fit(X_train,target)
    for i in model.coef_:
        if i != 0:
            zero_flag=False
            break
    if zero_flag == True:
        print('All zero coef!')
        model = linear_model.Lasso(alpha=0.1)
        model.fit(X_train,target)
    return model.coef_

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten

def Uni_NN(X_train,target,testX, testy,epoch,hidden_layers=1):
    inputs = Input(shape=X_train[0].shape, name='main_input')
    main_branch = Dense(200)(inputs)
    main_branch = Activation("relu")(main_branch)
    #NN layer
    for _ in range(hidden_layers):
        main_branch = Dense(200, activation='relu')(main_branch)
    #res
    main_branch = Dense(100, activation='relu')(main_branch)
    res = Dense(1, activation='relu', name='res')(main_branch)
    model = Model(inputs = inputs,
         outputs = [res])
    #model.summary()
    opt = keras.optimizers.Adam(lr=0.01)
    model.compile(optimizer=opt,
              loss={'res': 'mean_absolute_error'},
              loss_weights={'res': 1})
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    model.fit({'main_input': X_train},
              {'res': target},
              epochs=epoch, batch_size=4,
              verbose=0,validation_data=(testX, testy),callbacks=[es]
             )
    print('fininsh')
    return model

5-fold 

In [ ]:

    
def get_new_X(x,beta,index):
    res = []
    for i in x:
        tmp_res = []
        tmp = i*beta
        for idx in index:
            tmp_res.append(tmp[idx])
        tmp_res = np.array(tmp_res)
        res.append(tmp_res)
    return np.array(res)
        

In [ ]:
import numpy as np
#     core_Non_zero_index = []
#     core_beta = np.zeros(74)
#     c_core_Non_zero_index = []
#     c_core_beta = np.zeros(74)
#     s_Non_zero_index = []
#     s_beta = np.zeros(74)
#     a_Non_zero_index = []
#     a_beta = np.zeros(74)
#     pa_Non_zero_index = []
#     pa_beta = np.zeros(74)
#     sa_Non_zero_index = []
#     sa_beta = np.zeros(74)
#     d_Non_zero_index = []
#     d_beta = np.zeros(74)
#     ts_Non_zero_index = []
#     ts_beta = np.zeros(74)
def get_coef_of_Lasso(x,h):
    Non_zero_index = []
    beta = np.zeros(74)
    # get beta of each factor

    #train
    #get core alpha
    beta = Lasso_Regression_alpha(x,h)
    for f in range(len(beta)):
        if beta[f] != 0:
            if f not in Non_zero_index:
                Non_zero_index.append(f)
                beta[f] = beta[f]
    #print('core_Non_zero_index',len(core_Non_zero_index),core_Non_zero_index)
    
    return  Non_zero_index , beta

In [ ]:
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
kf = KFold(n_splits=5, random_state=None, shuffle=False)

core_mae = []
c_core_mae = []
s_mae = []
a_mae = []
pa_mae = []
sa_mae = []
d_mae = []
ts_mae = []
for train_index, test_index in kf.split(x):
    # 5-fold split test/train dataset
    X_test =  x[test_index]
    y_test =  label[test_index]
    
    X_train =  x[train_index]
    y_train =  label[train_index]
    #get_coef
    core_Non_zero_index , core_beta = get_coef_of_Lasso(X_train,y_train[:,0])
    print('core_Non_zero_index',len(core_Non_zero_index),core_Non_zero_index)
    c_core_Non_zero_index , c_core_beta = get_coef_of_Lasso(X_train,y_train[:,1])
    print('c_core_Non_zero_index',len(c_core_Non_zero_index),c_core_Non_zero_index)
    s_Non_zero_index , s_beta = get_coef_of_Lasso(X_train,y_train[:,2])
    print('s_Non_zero_index',len(s_Non_zero_index),s_Non_zero_index)
    a_Non_zero_index , a_beta = get_coef_of_Lasso(X_train,y_train[:,3])
    print('a_Non_zero_index',len(a_Non_zero_index),a_Non_zero_index)
    pa_Non_zero_index , pa_beta = get_coef_of_Lasso(X_train,y_train[:,4])
    print('pa_Non_zero_index',len(pa_Non_zero_index),pa_Non_zero_index)
    sa_Non_zero_index , sa_beta = get_coef_of_Lasso(X_train,y_train[:,5])
    print('sa_Non_zero_index',len(sa_Non_zero_index),sa_Non_zero_index)
    d_Non_zero_index , d_beta = get_coef_of_Lasso(X_train,y_train[:,6])
    print('d_Non_zero_index',len(d_Non_zero_index),d_Non_zero_index)
    ts_Non_zero_index , ts_beta = get_coef_of_Lasso(X_train,y_train[:,7])
    print('ts_Non_zero_index',len(ts_Non_zero_index),ts_Non_zero_index)
    #training and evaluate
    epoch = 5
    #epoch = 3
    #core
    if len(core_Non_zero_index)!=0:
        core_train = y_train[:,0]
        core_X_train = get_new_X(X_train,core_beta,core_Non_zero_index)
        core_test = y_test[:,0]
        core_X_test = get_new_X(X_test,core_beta,core_Non_zero_index)
        model1 = Uni_NN(core_X_train,core_train,core_X_test,core_test,epoch )
        core_rmae = mean_absolute_error( core_test, model1.predict( core_X_test))
        core_mae.append(core_rmae)
        
    else:
        core_test = y_test[:,0]
        core_rmae = mean_absolute_error( core_test, np.zeros(len(core_test)))
        core_mae.append(core_rmae)
    #c_core
    if len(c_core_Non_zero_index)!=0:
        c_core_train = y_train[:,1]
        c_core_X_train = get_new_X(X_train,c_core_beta,c_core_Non_zero_index)
        c_core_test = y_test[:,1]
        c_core_X_test = get_new_X(X_test,c_core_beta,c_core_Non_zero_index)
        model2 = Uni_NN(c_core_X_train,c_core_train,c_core_X_test,c_core_test,epoch )
        c_core_rmae = mean_absolute_error( c_core_test, model2.predict( c_core_X_test))
        c_core_mae.append(c_core_rmae)
    else:
        c_core_test = y_test[:,1]
        c_core_rmae = mean_absolute_error( c_core_test, np.zeros(len(c_core_test)))
        c_core_mae.append(c_core_rmae)
    #s
    if len(s_Non_zero_index)!=0:
        s_train = y_train[:,2]
        s_X_train = get_new_X(X_train,s_beta,s_Non_zero_index)
        s_test = y_test[:,2]
        s_X_test = get_new_X(X_test,s_beta,s_Non_zero_index)
        model3 = Uni_NN(s_X_train,s_train,s_X_test,s_test,epoch )
        s_rmae = mean_absolute_error( s_test, model3.predict( s_X_test))
        s_mae.append(s_rmae)
    else:
        s_test = y_test[:,2]
        s_rmae = mean_absolute_error( s_test, np.zeros(len(s_test)))
        s_mae.append(s_rmae)
    #a
    if len(a_Non_zero_index)!=0:
        a_train = y_train[:,3]
        a_X_train = get_new_X(X_train,a_beta,a_Non_zero_index)
        a_test = y_test[:,3]
        a_X_test = get_new_X(X_test,a_beta,a_Non_zero_index)
        model4 = Uni_NN(a_X_train,a_train,a_X_test,a_test,epoch )
        a_rmae = mean_absolute_error( a_test, model4.predict( a_X_test))
        a_mae.append(a_rmae)
    else:
        a_test = y_test[:,3]
        a_rmae = mean_absolute_error( a_test, np.zeros(len(a_test)))
        a_mae.append(a_rmae)
    #pa
    if len(pa_Non_zero_index)!=0:
        pa_train = y_train[:,4]
        pa_X_train = get_new_X(X_train,pa_beta,pa_Non_zero_index)
        pa_test = y_test[:,4]
        pa_X_test = get_new_X(X_test,pa_beta,pa_Non_zero_index)
        model5 = Uni_NN(pa_X_train,pa_train,pa_X_test,pa_test,epoch )
        pa_rmae = mean_absolute_error( pa_test, model5.predict( pa_X_test))
        pa_mae.append(pa_rmae)
    else:
        pa_test = y_test[:,4]
        pa_rmae = mean_absolute_error( pa_test, np.zeros(len(pa_test)))
        pa_mae.append(pa_rmae)
    #sa
    if len(sa_Non_zero_index)!=0:
        sa_train = y_train[:,5]
        sa_X_train = get_new_X(X_train,sa_beta,sa_Non_zero_index)
        sa_test = y_test[:,5]
        sa_X_test = get_new_X(X_test,sa_beta,sa_Non_zero_index)
        model6 = Uni_NN(sa_X_train,sa_train,sa_X_test,sa_test,epoch )
        sa_rmae = mean_absolute_error( sa_test, model6.predict( sa_X_test))
        sa_mae.append(sa_rmae)
    else:
        sa_test = y_test[:,5]
        sa_rmae = mean_absolute_error( sa_test, np.zeros(len(sa_test)))
        sa_mae.append(sa_rmae)
    #d
    if len(d_Non_zero_index)!=0:
        d_train = y_train[:,6]
        d_X_train = get_new_X(X_train,d_beta,d_Non_zero_index)
        d_test = y_test[:,6]
        d_X_test = get_new_X(X_test,d_beta,d_Non_zero_index)
        model7 = Uni_NN(d_X_train,d_train,d_X_test,d_test,epoch )
        d_rmae = mean_absolute_error( d_test, model7.predict( d_X_test))
        d_mae.append(d_rmae)
    else:
        d_test = y_test[:,6]
        d_rmae = mean_absolute_error( d_test, np.zeros(len(d_test)))
        d_mae.append(d_rmae)
    #ts
    if len(ts_Non_zero_index)!=0:
        ts_train = y_train[:,7]
        ts_X_train = get_new_X(X_train,ts_beta,ts_Non_zero_index)
        ts_test = y_test[:,7]
        ts_X_test = get_new_X(X_test,ts_beta,ts_Non_zero_index)
        model8 = Uni_NN(ts_X_train,ts_train,ts_X_test,ts_test,epoch )
        ts_rmae = mean_absolute_error( ts_test, model8.predict( ts_X_test))
        ts_mae.append(ts_rmae)
    else:
        ts_test = y_test[:,7]
        ts_rmae = mean_absolute_error( ts_test, np.zeros(len(ts_test)))
        ts_mae.append(ts_rmae)

#show result
core_mean = sum(core_mae) / len(core_mae)
core_std = np.std(core_mae)
print('core: %.2f ± %.2f' % (core_mean, core_std))
c_core_mean = sum(c_core_mae) / len(c_core_mae)
c_core_std = np.std(c_core_mae)
print('c_core: %.2f ± %.2f' % (c_core_mean, c_core_std))
s_mean = sum(s_mae) / len(s_mae)
s_std = np.std(s_mae)
print('s: %.2f ± %.2f' % (s_mean, s_std))
a_mean = sum(a_mae) / len(a_mae)
a_std = np.std(a_mae)
print('a: %.2f ± %.2f' % (a_mean, a_std))
pa_mean = sum(pa_mae) / len(pa_mae)
pa_std = np.std(pa_mae)
print('pa: %.2f ± %.2f' % (pa_mean, pa_std))
sa_mean = sum(sa_mae) / len(sa_mae)
sa_std = np.std(sa_mae)
print('sa: %.2f ± %.2f' % (sa_mean, sa_std))
d_mean = sum(d_mae) / len(d_mae)
d_std = np.std(d_mae)
print('d: %.2f ± %.2f' % (d_mean, d_std))
ts_mean = sum(ts_mae) / len(ts_mae)
ts_std = np.std(ts_mae)
ts_std = np.std(ts_mae)
print('ts: %.2f ± %.2f' % (ts_mean, ts_std))

Multi-task learning

In [ ]:
Avg_mae = np.concatenate([core_mae,c_core_mae,s_mae,a_mae,pa_mae,sa_mae,d_mae,ts_mae])
#print(Avg_mae)
print('Avg: %.2f ± %.2f' % (sum(Avg_mae)/len(Avg_mae),np.std(Avg_mae)))

draw model structure

In [ ]:
Avg_mae = np.concatenate([core_mae,c_core_mae,s_mae,a_mae,pa_mae,sa_mae,d_mae])
#print(Avg_mae)
print('Avg: %.2f ± %.2f' % (sum(Avg_mae)/len(Avg_mae),np.std(Avg_mae)))

training